# Style Transfer Using Trained CNN On Spectrograms:

# Training a CNN on Image Spectrograms:

In [ ]:
% run audio_utils.ipynb
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import tensorflow as tf
import os
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Data Preprocessing:
### We first trained a CNN on audio classification using  dataset from https://datahack.analyticsvidhya.com/contest/practice-problem-urban-sound-classification/

### Converting raw audio samples from .wav to 2-D spectrograms

In [ ]:
listofwav=[]

def load_wav(path):
    for dirname, subdir, filelist in os.walk(path):
        for filename in filelist:
            if '.wav' in filename.lower():
                listofwav.append(os.path.join(dirname,filename))
    return listofwav

In [ ]:
audio_utils = AudioUtils()

In [ ]:
w = np.load('saved_weights.npy')

In [ ]:
w.shape

In [ ]:
X=[]
y_index = []
wav = load_wav('train/Train/')
for i in range(len(listofwav)):
    print(i)
    temp = audio_utils.wave_to_spectrogram(listofwav[i])[0]
    if temp.shape[1] == 173:
        X.append(temp)
        y_index.append(i)

In [ ]:
X_data = np.zeros((len(X),1025,173,1))

In [ ]:
for i in range(len(X)):
    X[i] = X[i].reshape(X[i].shape[0], X[i].shape[1], 1)
    X_data[i] = X[i]

In [ ]:
np.save('training_spectrograms_X',X_data)

In [ ]:
X_data = np.load('training_spectrograms_X.npy')

In [ ]:
X_data_1 = X_data[:100]

### Training labels

In [ ]:
import pandas as pd
train = pd.read_csv('train/train.csv')

In [ ]:
def parser(row):
    label = row.Class
    return label

In [ ]:
temp = train.apply(parser, axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
y = np.array(temp.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(y))

In [ ]:
y_data=np.zeros((len(y_index),10))

In [ ]:
for i in range(len(y_index)):
    y_data[i] = y[y_index[i]]

In [ ]:
y_data = np.load('training_y.npy')

In [ ]:
y_data_1 = y_data[:100]

In [ ]:
np.save('y_data_1',y_data_1)
np.save('X_data_1',X_data_1)

## Training Sequential Model using Keras


In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(5,kernel_size = 3 ,activation = 'relu',input_shape=(1025,173,1)))

In [ ]:
model.add(Conv2D(10,kernel_size = 3, activation = 'relu'))

In [ ]:
model.add(Conv2D(20,kernel_size = 3, activation = 'relu'))

In [ ]:
model.add(Conv2D(30,kernel_size = 3, activation = 'relu'))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(1024, activation = 'relu'))

In [ ]:
model.add(Dense(512, activation = 'relu'))

In [ ]:
model.add(Dense(10, activation = 'softmax'))

In [ ]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy')

In [ ]:
model.fit(X_data_1,y_data_1,epochs = 10,batch_size=32)

In [ ]:
X_data.shape

In [ ]:
y_data.shape

In [ ]:
content_filename = "/Users/durveshvedak/Desktop/Fall 2018/Deep Learning/AudioTrainedNetwork/Results/Shallow CNN/Result 4/laser.wav"
style_filename = "applause2.wav"

In [ ]:
display(Audio(content_filename))
display(Audio(style_filename))

In [ ]:
audio_utils = AudioUtils()
a_content, sampling_rate = audio_utils.wave_to_spectrogram(content_filename,2048)
a_style, sampling_rate = audio_utils.wave_to_spectrogram(style_filename,2048)

In [ ]:
n_samples_time = a_content.shape[1] # considering time domain as samples
n_channels_frequency = a_content.shape[0] # considering frequency domain as channels
a_style = a_style[:n_channels_frequency, :n_samples_time] # making sure style and content tensors have same shape

In [ ]:
a_style.shape

In [ ]:
#plotting style and content spectrogram
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Content')
plt.imshow(a_content[:400,:])
plt.subplot(1, 2, 2)
plt.title('Style')
plt.imshow(a_style[:300,:])
plt.show()

In [ ]:
a_style.shape

In [ ]:
x = np.random.randn(1,a_content.shape[0],a_content.shape[1],1)

In [ ]:
a_content = a_content.reshape((1,a_content.shape[0],a_content.shape[1],1))

In [ ]:
a_content_tf = np.ascontiguousarray(a_content)

In [ ]:
a_style = a_style.reshape((1,a_style.shape[0],a_style.shape[1],1))

In [ ]:
a_style_tf = np.ascontiguousarray(a_style)

In [ ]:
layer_1 = np.load('layer_1.npy')
layer_2 = np.load('layer_2.npy')
layer_3 = np.load('layer_3.npy')

In [ ]:
kernel = layer_1

In [ ]:
kernel_2 = layer_2

In [ ]:
g = tf.Graph()
with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
    y = tf.placeholder('float32', [1,n_channels_frequency,n_samples_time,1], name="y")
    kernel_tf = tf.constant(kernel, name="kernel", dtype='float32')
    conv = tf.nn.conv2d(
            y,
            kernel_tf,
            strides=[1, 1, 1, 1],
            padding="VALID",
            name="conv")
    
    kernel_tf_2 = tf.constant(kernel_2, name="kernel_2", dtype='float32')
    
    conv_2 = tf.nn.conv2d(
            conv,
            kernel_tf_2,
            strides=[1, 1, 1, 1],
            padding="VALID",
            name="conv_2")
    
    net = tf.nn.relu(conv_2)
    content_features = net.eval(feed_dict={y: a_content})
    style_features = net.eval(feed_dict={y: a_style})

In [ ]:
style_features.shape

In [ ]:
features = np.reshape(style_features,(style_features.shape[1] * style_features.shape[2],10))

In [ ]:
style_gram = np.matmul(features.T, features) 

In [ ]:
result = None
with tf.Graph().as_default():
    x = x = tf.Variable(a_style, name="x")
    #x = tf.Variable(np.random.randn(1,n_channels_frequency,n_samples_time,1).astype(np.float32)*1e-3, name="x")
    
    kernel_tf = tf.constant(kernel, name="kernel", dtype='float32')
    conv = tf.nn.conv2d(
        x,
        kernel_tf,
        strides=[1, 1, 1, 1],
        padding="VALID",
        name="conv")
    
    kernel_tf_2 = tf.constant(kernel_2, name="kernel_2", dtype='float32')
    
    conv_2 = tf.nn.conv2d(
            conv,
            kernel_tf_2,
            strides=[1, 1, 1, 1],
            padding="VALID",
            name="conv_2")
    
    net = tf.nn.relu(conv_2)
    content_loss = tf.nn.l2_loss(
            net - content_features)

    style_loss = 0


    feats = tf.reshape(net,(style_features.shape[1] * style_features.shape[2],10))
    gram = tf.matmul(tf.transpose(feats), feats) 
    gram = gram * 0.001
    style_loss = tf.nn.l2_loss(gram - style_gram)

     # Overall loss
    loss =  content_loss + style_loss

    #opt = tf.contrib.opt.ScipyOptimizerInterface(
          #loss, method='L-BFGS-B', options={'maxiter': 10000})
    opt = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    # Optimization
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        # initial = x.eval()
        # initial = initial.reshape(n_channels_frequency,n_samples_time)
        
        print('Started optimization.')
        for i in range(5000):
            print(i)
            sess.run(opt)
            print(sess.run(loss))
            
        #opt.minimize(sess)

        print ('Final loss:', loss.eval())
        result = x.eval()

In [ ]:
result = result.reshape(n_channels_frequency,n_samples_time)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Result')
plt.imshow(result[:400,:])

In [ ]:
audio_utils = AudioUtils()
a_content, sampling_rate = audio_utils.wave_to_spectrogram(content_filename,2048)
a_style, sampling_rate = audio_utils.wave_to_spectrogram(style_filename,2048)

In [ ]:
n_samples_time = a_content.shape[1] # considering time domain as samples
n_channels_frequency = a_content.shape[0] # considering frequency domain as channels
a_style = a_style[:n_channels_frequency, :n_samples_time] # making sure style and content tensors have same shape

In [ ]:
audio_utils.spectrogram_to_wave(result , 2048 , "out3.wav", sampling_rate)

In [ ]:
display(Audio("out3.wav"))

In [ ]:
display(Audio(content_filename))
display(Audio(style_filename))

In [ ]:
sum(sum(np.square(result - a_content)))

In [ ]:
sum(sum(np.square(result - a_style)))

In [ ]:
a_style, sampling_rate = audio_utils.wave_to_spectrogram('/Users/durveshvedak/PycharmProjects/DeepAudioStyleTransfer/out.wav',2048)
a_content, sampling_rate = audio_utils.wave_to_spectrogram('/Users/durveshvedak/Desktop/Fall 2018/Deep Learning/AudioTrainedNetwork/applause2.wav',2048)

In [ ]:
n_samples_time = a_content.shape[1] # considering time domain as samples
n_channels_frequency = a_content.shape[0] # considering frequency domain as channels
a_style = a_style[:n_channels_frequency, :n_samples_time] # making sure style and content tensors have same shape

In [ ]:

plt.title('Result')
plt.imshow(a_style[:400,:])
plt.show()